In [1]:
%run logs.ipynb

In [2]:
def simulate_multiple_assets_vault(assets_df_list=[], assets_ratio=[], starting_buying_power=2000,
                                   slippage=0.05, sell_apr_percentage = 0.01, rebalance_interval_days=7,
                                   stop_sell=0.5, buying_power_stoploss=0.3):
    """Simulate model for multiple assets.
    
    The function simulates the behaviour of our vault wrt to the
    changes in assets that were combined to make the vault.

    :param assets_df_list: List of pandas dataframe that make up the
        vault
    :param assets_ratio: List of how much each asset contributes to
        vault
    :param starting_buying_power: Initail buying power of the vault
    :param slippage: Slippage to consider while executing any trade
    :param sell_apr_percentage: Percent of appreciation to sell
    :rebalance_interval_days: Number of days after which rebalancing
        should happen
    :stop_sell: Stop selling when the vault drop below this percent
    :buying_power_stoploss: Stop when buying power of any asset drops
        below this percent
    :return: Buying power, held quantity, price of assets and vault
    :rtype: dict
    """
    
    assert(len(assets_df_list) == len(assets_ratio))
    assert(sum(assets_ratio) == 1)
    
    # Get no. of assets.
    no_of_assets = len(assets_df_list)
    
    # Calculate previous/initial price for all assets.
    assets_previous_price = [asset_df.iloc[0, 1] for asset_df in assets_df_list]
    min_assets_balance_allowed = []
    
    slippage_multiplier = (1 - slippage)
    counter = 0
    
    simulation_logs = {
        'no_of_assets': no_of_assets,
        'date': [],
        'vault': [],
        'vault_buying_power': [],
    }
    
    vault_status = {
        'vault_buying_power': starting_buying_power,
    }
    
    for i in range(no_of_assets):
        column_name = assets_df_list[i].columns[1]
        desired_column_name = column_name.split('_')[-1]
        
        simulation_logs[f'asset_{i+1}'] = []
        simulation_logs[f'asset_{i+1}_name'] = desired_column_name
        simulation_logs[f'asset_{i+1}_buying_power'] = []
        simulation_logs[f'asset_{i+1}_price'] = []
        
        vault_status[f'asset_{i+1}'] = (assets_ratio[i] * starting_buying_power) / assets_previous_price[i]
        vault_status[f'asset_{i+1}_buying_power'] = assets_ratio[i] * starting_buying_power
        vault_status[f'asset_{i+1}_name'] = desired_column_name
        
        min_assets_balance_allowed.append((1 - stop_sell) * vault_status[f'asset_{i+1}'])
    
    print('Start with vault as below:')
    pp.pprint(vault_status)
    print()
    
    for i in assets_df_list[0].index.to_list():
        index = i
        simulation_day_date = assets_df_list[0].loc[i, 'Date']
        assets_price = [asset_df.iloc[i, 1] for asset_df in assets_df_list]
        
        def skip():
            simulation_logs['date'].append(simulation_day_date)
            simulation_logs['vault_buying_power'].append(new_buying_power)
            
            for i in range(no_of_assets):
                simulation_logs[f'asset_{i+1}'].append(vault_status[f'asset_{i+1}'])
                simulation_logs[f'asset_{i+1}_price'].append(assets_price[i])
                simulation_logs[f'asset_{i+1}_buying_power'].append(new_assets_buying_power[i])
            
            assets_previous_price = assets_price[:]
            
        # Calculate the old buying power of individual and overall of all assets.
        old_assets_buying_power = [vault_status[f'asset_{i+1}_buying_power'] for i in range(no_of_assets)]
        old_buying_power = sum(old_assets_buying_power)

        # Calculate the new balance & buying power of individual and overall of all assets.
        new_assets_balance = [vault_status[f'asset_{i+1}'] for i in range(no_of_assets)]
        new_assets_buying_power = [new_assets_balance[i] * assets_price[i] for i in range(no_of_assets)]
        new_buying_power = sum(new_assets_buying_power)
        
        # Update the buying power of vault.
        for i in range(no_of_assets):
            vault_status[f'asset_{i+1}_buying_power'] = new_assets_buying_power[i]    
        
        # Now the target buying power should be something that maintains the ratio we have for each asset.
        target_assets_buying_power = [new_buying_power * assets_ratio[i] for i in range(no_of_assets)]

        # Caclulate the difference in current and target buying power.
        diff_assets_buying_power = [
            abs(new_assets_buying_power[i] - target_assets_buying_power[i]) 
            for i in range(no_of_assets)
        ]
        
        # Strategy #1 -> Sell the most appreciated asset for the rest.
        #     - Find the most appreciated Asset.
        #     - Decide how much to sell of that asset? (i.e How much of the appreciated value)
        #     - Split that amount across all the other assets to buy them.
        #     eg: before ({A: 100, B: 100, C: 100, D: 100}) -> 400.
        #         now    ({A: 150, B: 200, C: 90, D: 50 })  -> 490 (+90).
        #         after  ({A: 172.5, B: 177.5, C: 112.5, D: 72.5 }) -> 490,
        #        (but split 25% of 90 across the others -> 22.5)
        #
        # 
        # Strategy #2 -> Sell all the appreciated assets for the rest in some combination..
        
        # Find the most appreciated asset.
        most_appreciated_asset_in_buying_power = -1 
        for i in range(no_of_assets):
            if (new_assets_buying_power[i] > target_assets_buying_power[i]):
                if (most_appreciated_asset_in_buying_power == -1):
                    most_appreciated_asset_in_buying_power = i 
                elif (diff_assets_buying_power[most_appreciated_asset_in_buying_power]
                      < diff_assets_buying_power[i]):
                    most_appreciated_asset_in_buying_power = i

        # Check if nothing has appreciated or not.
        nothing_appreciated_in_buying_power = most_appreciated_asset_in_buying_power != -1
        if not nothing_appreciated_in_buying_power:
            print('Nothing appreciated')
            skip()
            continue
        
        # Update the balance based on price and ratio of appreciated amount.
        appreciated_buying_power = diff_assets_buying_power[most_appreciated_asset_in_buying_power]
        for i in range(no_of_assets):
            new_assets_balance[i] += (appreciated_buying_power * assets_ratio[i]) / assets_price[i] * slippage_multiplier
            
        # Check if we are hitting our stop sell or not.
        stopsell_hit = False
        for i in range(no_of_assets):
            if new_assets_balance[i] <= min_assets_balance_allowed[i]:
                stopsell_hit = True
                skip()
                break
        if stopsell_hit:
            print('Vault triggered a trade but was cancelled because the stop sell was hit')
            skip()
            continue
        
        # Check if we are trading in the right day or not.
        invalid_trading_day = index % rebalance_interval_days != 0
        if invalid_trading_day:
            print('Invalid trading day',index, index % rebalance_interval_days )
            skip()
            continue

        # Check if there is no change in the buying power
        no_change_in_buying_power = new_buying_power < old_buying_power
        if no_change_in_buying_power:
            print('No change in buying power.')
            skip()
            continue
        
        # Check if we are selling too much of a particular asset. Ideally we should make
        # sure that we buy/sell as much so that the underlying does not get fully sold off.
        selling_too_much = False
        for i in range(no_of_assets):
            if new_assets_balance[i] <= 0:
                selling_too_much = True
                skip()
                break
        if selling_too_much:
            print('Vault triggered a trade but was cancelled because we\'d have fully sold one asset')
            continue
        
        # Check if our stop loss has been hit or not, in which case we bail.
        if (new_buying_power - starting_buying_power) / starting_buying_power <= buying_power_stoploss * -1: 
            print('Breaking')
            break

        # Recalculate the new vault's status.
        counter += 1
        for i in range(no_of_assets):
            vault_status[f'asset_{i+1}'] = new_assets_balance[i]
            vault_status[f'asset_{i+1}_buying_power'] = new_assets_balance[i] * assets_price[i]
            vault_status['vault_buying_power'] = new_buying_power
        
        simulation_logs['date'].append(simulation_day_date)
        
        buying_power_accumulator = 0
        for i in range(no_of_assets):
            simulation_logs[f'asset_{i+1}'].append(vault_status[f'asset_{i+1}'])
            simulation_logs[f'asset_{i+1}_price'].append(assets_price[i])
            simulation_logs[f'asset_{i+1}_buying_power'].append(vault_status[f'asset_{i+1}_buying_power'])
            buying_power_accumulator += vault_status[f'asset_{i+1}_buying_power']
        
        simulation_logs['vault_buying_power'].append(buying_power_accumulator)
        assets_previous_price = assets_price[:]

    print("Finished with %d rebalances" % counter)
    print("Final buying power of the vault is %d" % (new_buying_power))
    print()
    
    print('Ending with vault as below:')
    pp.pprint(vault_status)
    
    return simulation_logs